# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10
build_tree = False
sample_batch_size=2000

In [2]:
# Parameters
cases_dir = "cases/case-100"
iterations = 3
number_samples = 100
build_tree = True


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

/tmp/ipykernel_26656/2173034037.py:4: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 100/100000 samples to cases/case-100/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.8.1


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores,
                                             sample_batch_size=sample_batch_size)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 100 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1665115566.307027']


Creating new index: [gdi init cases/case-100/index]


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f7d90ffa160>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f7d90ffa160>
  self.pid = os.fork()


Creating a new index took 3.41 seconds
Analysis running: [gdi --project-dir cases/case-100/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-100/input-files-case.tsv]


...

/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f7d90ffa160>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f7d90ffa160>
  self.pid = os.fork()


Building tree took 0.25 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f7ef67528b0> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


In [7]:
benchmark_df

,Name,Reference name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),...,Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,case-100,NC_045512,1,100,16953,769,32,29903,188.86,2.553348e+09,...,3.780866e+09,12.94,6.264758e+09,344752128.0,4378624.0,13.86,301568000.0,251985920.0,215.66,6.264758e+09
0,case-100,NC_045512,2,100,16953,769,32,29903,172.24,3.252081e+09,...,3.780878e+09,12.53,6.258020e+09,345645056.0,4374528.0,15.14,302026752.0,252325888.0,199.91,6.258020e+09
0,case-100,NC_045512,3,100,16953,769,32,29903,175.15,3.206930e+09,...,3.780874e+09,12.60,6.256128e+09,344195072.0,4374528.0,14.74,302104576.0,252293120.0,202.49,6.256128e+09


In [8]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [9]:
if build_tree:
    !gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
    print(f'Wrote tree to {output_tree}')
else:
    print(f'build_tree={build_tree} so no tree to export')

Wrote tree to cases/case-100/tree.tre
